In [ ]:
%pwd #Đang đứng ở /content

'/content'

In [ ]:
import os
os.chdir('../')

In [ ]:
%pwd

In [ ]:
!pip install pi-heif "unstructured[local-inference,pdf]" pdfminer.six pypdf Pillow
!pip install -U langchain-community langchain-core
!pip install -U langchain-text-splitters
!pip install langchain==0.3.27
!pip install qdrant-client
!pip install unstructured
!pip install python-docx



#Extract PDF

In [ ]:
from langchain_community.document_loaders import DirectoryLoader, UnstructuredPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


def load_pdf_files(data):
    loader = DirectoryLoader(
        path=data,
        glob="*.pdf",
        loader_cls=UnstructuredPDFLoader
    )
    documents = loader.load()
    return documents

#Extract .txt

In [16]:
from langchain.document_loaders import DirectoryLoader , UnstructuredWordDocumentLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


def load_word_files(data):
    loader = DirectoryLoader(
        path=data,
        glob="*.docx",
        loader_cls=UnstructuredWordDocumentLoader #loại file cần load
    )
    documents = loader.load()
    return documents

In [17]:
extracted_data = load_word_files(data='DataChatbot')

In [18]:
extracted_data

[Document(metadata={'source': 'DataChatbot/dataset_skin_desease.docx'}, page_content='1. U mạch sừng hóa (Angiokeratoma)\n\nU mạch sừng hóa (Angiokeratoma) là một tình trạng da lành tính, đặc trưng bởi các tổn thương nhỏ, sần sùi, màu đỏ hoặc tím sẫm. Chúng hình thành do sự giãn nở của các mạch máu nhỏ (mao mạch) nằm sát bề mặt da, cùng với sự dày lên bất thường của lớp biểu bì. \n\nĐặc điểm nhận biết \n\nKích thước: Các nốt thường nhỏ, dưới 1 cm.\n\nMàu sắc: Thay đổi từ đỏ tươi, đỏ sẫm đến tím hoặc gần như đen.\n\nBề mặt: Sần sùi, thô ráp hoặc có vảy giống mụn cóc.\n\nVị trí: Có thể xuất hiện ở nhiều vùng khác nhau trên cơ thể, tùy thuộc vào loại u mạch sừng hóa.\n\nTriệu chứng: Thường không gây đau hay khó chịu. Tuy nhiên, các nốt này có thể dễ bị chảy máu nếu bị cào, chà xát hoặc bị tổn thương. \n\nPhân loại\n\nCó nhiều loại u mạch sừng hóa, bao gồm: \n\nU mạch sừng hóa Fordyce: Phổ biến nhất, thường xuất hiện ở bìu (đàn ông) hoặc âm hộ (phụ nữ), với các nốt nhỏ, đỏ sẫm.\n\nU mạch s

In [ ]:
extracted_data = load_pdf_files(data='DataChatbot')   #Đọc các file trong folder này

In [ ]:
extracted_data

[Document(metadata={'source': 'DataChatbot/Huong-dan-chan-doan-dieu-tri-Da-lieu.pdf'}, page_content='BỘ Y TẾ\n\nHƯỚNG DẪN CHẨN ĐOÁN VÀ ĐIỀU TRỊ CÁC BỆNH DA LIỄU (ban hành kèm theo Quyết định số 75/QĐ-BYT ngày 13/01/2015)\n\nHà Nội, 2015\n\n1\n\nBỘ Y TẾ\n\nCỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM Độc lập - Tự do - Hạnh phúc\n\nSố: 75/QĐ-BYT\n\nHà Nội, ngày 13 tháng 01 năm 2015\n\nQUYẾT ĐỊNH Về việc ban hành tài liệu chuyên môn “Hướng dẫn chẩn đoán và điều trị các bệnh da liễu”\n\nBỘ TRƯỞNG BỘ Y TẾ\n\nCăn cứ Luật khám bệnh, chữa bệnh năm 2009;\n\nCăn cứ Nghị định số 63/2012/NĐ-CP ngày 31/8/2012 của Chính Phủ quy\n\nđịnh chức năng, nhiệm vụ, quyền hạn và cơ cấu tổ chức của Bộ Y tế;\n\nTheo đề nghị của Cục trưởng Cục Quản lý Khám, chữa bệnh,\n\nQUYẾT ĐỊNH:\n\nĐiều 1. Ban hành kèm theo Quyết định này tài liệu chuyên môn “Hướng dẫn\n\nchẩn đoán và điều trị các bệnh da liễu”.\n\nĐiều 2. Tài liệu “Hướng dẫn chẩn đoán và điều trị các bệnh da liễu” ban hành kèm theo Quyết định này được áp dụng tại các

Preprocessing (Tiền xử lý dữ liệu)

In [ ]:
import re

def preprocess_data(text):


    #Xóa URL
    text = re.sub(r'(https?://\S+|www\.\S+)', '', text)
    # Xoá các dòng chỉ chứa dấu = hoặc -
    text = re.sub(r'^[=\-]{2,}\s*$', '', text, flags=re.MULTILINE)
    #Xoá các ký tự bảng markdown (|, ---)
    text = re.sub(r'\|.*?\|', '', text)
    # Xoá emoji và ký tự Unicode không cần thiết
    text = re.sub(r'[^\w\s,.!?à-ỹÀ-Ỹ\-–]', '', text)
    # Xoá khoảng trắng thừa và dòng trống
    text = re.sub(r'\s+', ' ', text).strip()
    return text


In [ ]:
from langchain_core.documents import Document

cleaned_data = []
for doc in extracted_data:
    cleaned_content = preprocess_data(doc.page_content)
    cleaned_doc = Document(
        page_content=cleaned_content,
        metadata=doc.metadata
    )
    cleaned_data.append(cleaned_doc)

In [ ]:
cleaned_data

[Document(metadata={'source': 'DataChatbot/Huong-dan-chan-doan-dieu-tri-Da-lieu.pdf'}, page_content='BỘ Y TẾ HƯỚNG DẪN CHẨN ĐOÁN VÀ ĐIỀU TRỊ CÁC BỆNH DA LIỄU ban hành kèm theo Quyết định số 75QĐ-BYT ngày 13012015 Hà Nội, 2015 1 BỘ Y TẾ CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM Độc lập - Tự do - Hạnh phúc Số 75QĐ-BYT Hà Nội, ngày 13 tháng 01 năm 2015 QUYẾT ĐỊNH Về việc ban hành tài liệu chuyên môn Hướng dẫn chẩn đoán và điều trị các bệnh da liễu BỘ TRƯỞNG BỘ Y TẾ Căn cứ Luật khám bệnh, chữa bệnh năm 2009 Căn cứ Nghị định số 632012NĐ-CP ngày 3182012 của Chính Phủ quy định chức năng, nhiệm vụ, quyền hạn và cơ cấu tổ chức của Bộ Y tế Theo đề nghị của Cục trưởng Cục Quản lý Khám, chữa bệnh, QUYẾT ĐỊNH Điều 1. Ban hành kèm theo Quyết định này tài liệu chuyên môn Hướng dẫn chẩn đoán và điều trị các bệnh da liễu. Điều 2. Tài liệu Hướng dẫn chẩn đoán và điều trị các bệnh da liễu ban hành kèm theo Quyết định này được áp dụng tại các cơ sở khám bệnh, chữa bệnh. Căn cứ vào tài liệu này và điều kiện cụ th

Chunking

In [ ]:
def text_split(cleaned_data):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=512,
        chunk_overlap=20,
        separators=["\n\n", "\n", ". ", " ", ""]
    )
    text_chunks = text_splitter.split_documents(cleaned_data)
    return text_chunks


In [ ]:
text_chunks = text_split(cleaned_data)
print(f'So luong chunk da tach', len(text_chunks))

So luong chunk da tach 1087


Embeddings vector -> (Vietnamese) 768-dimensional vector space

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

def download_huggingface_embeddings():
    embeddings = HuggingFaceEmbeddings(
        model_name="dangvantuan/vietnamese-embedding"
    )
    return embeddings

In [ ]:
embeddings = download_huggingface_embeddings()

In [ ]:
query_result = embeddings.embed_documents(["Hà Nội là thủ đô của Việt Nam", "Đà Nẵng là thành phố du lịch"])
print("Chiều của vector 1:", len(query_result[0])) #- > thấy là dù cho vui_tính có dấu _ SBERT này vẫn xử lý tốt 768 embeddings đúng

Chiều của vector 1: 768


In [ ]:
#Load .env co san len
from dotenv import load_dotenv
import os
load_dotenv()

True

In [ ]:
QDRANT_API_KEY=os.environ.get('QDRANT_API_KEY')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')

Tạo qdrant - và liên kết

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams
import os





client = QdrantClient(
    url="https://b6797286-70e3-43c5-91eb-4a88a7a227a0.us-west-1-0.aws.cloud.qdrant.io",
    api_key=QDRANT_API_KEY
)


collection_name = "medical-chatbot"

client.recreate_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(
        size=768,
        distance=Distance.COSINE  # metric cosine
    ),
)




/tmp/ipython-input-2754785471.py:17: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

In [ ]:
!pip install langchain-qdrant qdrant-client


Đưa chunk vào db

In [ ]:
from langchain_qdrant import QdrantVectorStore
from langchain_community.embeddings import HuggingFaceEmbeddings



embeddings = HuggingFaceEmbeddings(model_name="dangvantuan/vietnamese-embedding")

QDRANT_API_KEY = os.getenv("QDRANT_API_KEY")


collection_name = "medical-chatbot"

docsearch = QdrantVectorStore.from_documents(
    documents=text_chunks,
    embedding=embeddings,
    url="https://b6797286-70e3-43c5-91eb-4a88a7a227a0.us-west-1-0.aws.cloud.qdrant.io",
    api_key=QDRANT_API_KEY,
    collection_name=collection_name,
)


In [ ]:
#Load các vector db đã lưu
docsearch = QdrantVectorStore.from_existing_collection(
    embedding=embeddings,
    url="https://b6797286-70e3-43c5-91eb-4a88a7a227a0.us-west-1-0.aws.cloud.qdrant.io",
    api_key=QDRANT_API_KEY,
    collection_name=collection_name,
)



#NỮA XỬ LÝ CV ở đây

======

In [ ]:
docsearch

Retrieval

In [ ]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":40})  #search_type="similarity tìm kiếm theo cosin

In [ ]:
#retrieval lúc chưa có rerank


#symetric rag
retriever_docs = retriever.invoke("Da của tôi bị nổi mẩn đỏ?")
retriever_docs

[Document(metadata={'source': 'DataChatbot/Huong-dan-chan-doan-dieu-tri-Da-lieu.pdf', '_id': '5392895a-a4f1-4c25-ab8d-a55bfec9e3fb', '_collection_name': 'medical-chatbot'}, page_content='. 173 Tăng hoặc giảm sắc tố làm da thâm xạm hay tạo thành những đám loang lổ. Da quanh hốc mắt có thể bị viêm đỏ, lộn mi, chảy nƣớc mắt. Ở mặt có thể biến dạng do phù nề hoặc thâm nhiễm làm mất đi các nếp nhăn trên mặt, thay vào đó là những khối gồ lên trông nhƣ mặt sƣ tử. Thƣơng tổn niêm mạc viêm môi, viêm kết mạc, viêm miệng... Thƣơng tổn tóc, móng đa số trƣờng hợp có rụng tóc, thƣa lông mày, lông mi'),
 Document(metadata={'source': 'DataChatbot/Huong-dan-chan-doan-dieu-tri-Da-lieu.pdf', '_id': 'ab2e3aac-10b3-47bd-bab1-d6db2a694d52', '_collection_name': 'medical-chatbot'}, page_content='. Thƣơng tổn da đỏ, có thể là những đốm màu hồng nhỏ, ấn kính mất màu, ranh giới không rõ với da lành, thƣờng bắt đầu ở các nếp gấp rồi lan nhanh ra toàn bộ cơ thể. Trên nền da đỏ, bong vảy da nhƣ vảy phấn, vùng da dà

Cross-encoder (Re-rank)

In [ ]:

#Tải model
from sentence_transformers import CrossEncoder

cross_encoder = CrossEncoder('itdainb/PhoRanker')

def rerank_documents(query, docs, top_n):

    # Chuyển đổi tài liệu thành định dạng phù hợp với cross-encoder
    pairs = [[query, doc.page_content] for doc in docs]

    # Tính điểm cho từng cặp (truy vấn, tài liệu)
    scores = cross_encoder.predict(pairs)

    # Kết hợp điểm với tài liệu ,Sắp xếp tài liệu theo điểm số giảm dần
    reranked = sorted(zip(docs, scores), key=lambda x: x[1], reverse=True)

    return [doc for doc, _ in reranked[:top_n]]

Prompt LLM

In [ ]:
from langchain_community.chat_models import ChatOpenAI

llm = ChatOpenAI(
    model="openai/gpt-oss-20b:free",
    openai_api_key=OPENAI_API_KEY,
    openai_api_base="https://openrouter.ai/api/v1",
    temperature=0.4,
    max_tokens=2048
)

NỮA CÓ CV ĐỔI PROMPT LẠI

In [ ]:
#prompt có thêm <system>

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate




system_prompt = (
    "Bạn là một trợ lý ảo chuyên về lĩnh vực y tế và sức khỏe, "
    "có nhiệm vụ trả lời câu hỏi của người dùng dựa trên **các đoạn ngữ cảnh được cung cấp**. "
    "Chỉ sử dụng thông tin trong ngữ cảnh — không được suy đoán hoặc bịa thêm. "
    "Nếu ngữ cảnh không chứa thông tin cần thiết, hãy trả lời: 'Xin lỗi, tôi không có đủ thông tin để trả lời câu hỏi này.'\n\n"
    "Yêu cầu định dạng:\n"
    "- Trả lời bằng **tiếng Việt tự nhiên, rõ ràng**.\n"
    "- Trình bày ngắn gọn, tối đa 4 câu.\n"
    "- Nếu có thể, tóm tắt trọng tâm hoặc đưa ra hướng dẫn đơn giản.\n\n"
    "=== NGỮ CẢNH ===\n{context}\n\n"
    "=== CÂU HỎI ==="
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])


Kết quả khi không có rerank

In [ ]:
question_awnser_chain = create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retriever,question_awnser_chain) # kết quả model trả ra , llm , và prompt

In [ ]:
response = rag_chain.invoke( {"input" :"Da của tôi bị nổi mẩn ngứa ?"}) #5s -> để trả lời
print(response['answer'])

Da của bạn có thể là viêm da tiếp xúc dị ứng hoặc viêm da cơ địa, thường gây ngứa, mẩn đỏ và đôi khi có sẩn, bong vảy. Khi mẩn nở, có thể xuất hiện mụn nở hoặc sẩn nhỏ. Hạn chế tiếp xúc với chất gây dị ứng, bảo vệ da và bôi kem chống ngứa hoặc thuốc giảm đau tại chỗ. Nếu tình trạng kéo dài hoặc nặng, nên tới bác sĩ da liễu để chẩn đoán và điều trị chính xác.


Kết quả khi có rerank

In [ ]:
query = "Da của tôi bị nổi mẩn ngứa ?"
question_answer_chain = create_stuff_documents_chain(llm, prompt)

#Lấy top-k
similar_docs = retriever.get_relevant_documents(query)

#Re-rank
top_docs = rerank_documents(query, similar_docs, 3)

#Gọi LLM trực tiếp
response = question_answer_chain.invoke({
    "input": query,
    "context": top_docs
})
print(response)

Bạn có thể đang gặp một dạng bệnh da gây đỏ, ngứa, bong vảy và có thể xuất hiện chảy dịch hoặc loét.  
Để giảm ngứa và ngăn ngừa sẹo, tránh gãi, giữ da ẩm bằng kem dưỡng, và tránh tiếp xúc với ánh nắng mạnh.  
Nếu triệu chứng kéo dài hoặc làm ảnh hưởng tới giấc ngủ, tâm lý, hãy tham khảo bác sĩ da liễu để được chẩn đoán và điều trị kịp thời.
